<h2>Project 0: Julia Basics</h2>

<h3>Introduction</h3>

<p>In this project, you will write a function to compute Euclidean distances between sets of vectors. But before you get started, you need to check out your code onto whatever computer you want to use. </p>

<strong>How to submit:</strong> You can submit your code using the red <strong>Submit</strong> button above. This button will send any code below surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags below to the autograder, which will then run several tests over your code. By clicking on the <strong>Details</strong> dropdown next to the Submit button, you will be able to view your submission report once the autograder has completed running. This submission report contains a summary of the tests you have failed or passed, as well as a log of any errors generated by your code when we ran it.

Note that this may take a while depending on how long your code takes to run! Once your code is submitted you may navigate away from the page as you desire -- the most recent submission report will always be available from the Details menu.

<p><strong>Evaluation:</strong> Your code will be autograded for technical
correctness and--on some assignments--speed. Please <em>do not</em> change the names of any provided functions or classes within the code, or you will wreak havoc on the autograder. Furthermore, <em>any code not surrounded by <strong>#&lt;GRADED&gt;</strong><strong>#&lt;/GRADED&gt;</strong> tags will not be run by the autograder</em>. However, the correctness of your implementation -- not the autograder's output -- will be the final judge of your score.  If necessary, we will review and grade assignments individually to ensure that you receive due credit for your work.

<p><strong>Academic Integrity:</strong> We will be checking your code against other submissions in the class for logical redundancy. If you copy someone else's code and submit it with minor changes, we will know. These cheat detectors are quite hard to fool, so please don't try. We trust you all to submit your own work only; <em>please</em> don't let us down. If you do, we will pursue the strongest consequences available to us.

<p><strong>Getting Help:</strong> You are not alone!  If you find yourself stuck  on something, contact the course staff for help.  Office hours, section, and the <a href="https://piazza.com/class/icxgflcnpra3ko">Piazza</a> are there for your support; please use them.  If you can't make our office hours, let us know and we will schedule more.  We want these projects to be rewarding and instructional, not frustrating and demoralizing.  But, we don't know when or how to help unless you ask.  




<h3> Euclidean distances in Julia </h3>

<p>Many machine learning algorithms access their input data primarily through pairwise distances. It is therefore important that we have a fast function that computes pairwise (Euclidean) distances of input vectors. Assume we have $n$ data vectors $\vec x_1,\dots,\vec x_n\in{\cal R}^d$ and $m$ vectors $\vec z_1,\dots,z_m\in{\cal R}^d$. And let us define two matrices $X=[\vec x_1,\dots,\vec x_n]\in{\cal R}^{d\times n}$, where the $i^{th}$ column is a vector $\vec x_i$ and similarly $Z=[\vec z_1,\dots,\vec z_m]$. Our distance function takes as input the two matrices $X$ and $Z$ and outputs a matrix $D\in{\cal R}^{n\times m}$, where 
	$$D_{ij}=\sqrt{(\vec x_i-\vec z_j)^\top (\vec x_i-\vec z_j)}.$$
</p>

A naïve implementation to compute pairwise distances may look like the code below:

In [1]:
function l2distanceSlow(X,Z=[])
    if isempty(Z);Z=X;end;
    (n,d)=size(X); # dimension of X
    m=size(Z,1); # dimension of Z
    D=zeros(n,m); # allocate memory for the output matrix
    for i=1:n # loop over vectors in X
        for j=1:m # loop over vectors in Z
            D[i,j]=0.0; 
            for k=1:d # loop over dimensions
                D[i,j]=D[i,j]+(X[i,k]-Z[j,k])^2; # compute l2-distance between the ith and jth vector
            end
            D[i,j]=sqrt(D[i,j]); # take square root
        end
    end
    return(D)
end

Please read through the code carefully and make sure you understand it. It is perfectly correct and will produce the correct result ... eventually. To see what is wrong, run the following program in the Julia console:


In [2]:
function jitdilemma()
    X=rand(700,100);
    println("Running the naive version for the first time ...")
    tic();D=l2distanceSlow(X);tslow1=toc();
    println("Running the naive version again ...")
    tic();D=l2distanceSlow(X,X);tslow2=toc();
end
jitdilemma() # function call

Running the naive version for the first time ...
elapsed time: 1.37725219 seconds
Running the naive version again ...
elapsed time: 0.021516014 seconds


0.021516014

This code defines some random data in $X$ and $Z$ and computes the corresponding distance matrix $D$. The <em>tic,toc</em> statements time how long this takes. On my laptop it takes about 8s the first time, and 0.06s the second time. This is strange, shouldn't the two function calls be exactly equivalent?

The problem is that the distance function uses a programming style that is well suited for C++ or Java, but  <strong>not Julia!!</strong>. The reason this is happening is that Julia is compiled with a just-in-time-compiler (JIT-Compiler). In the second case it realizes that these operations can be vectorized, whereas in the first case case it doesn't. 
If the code cannot be compiled, it is interpreted with a lot of slow execution overhead. 
The difference in computation time is staggering. The problem is that we can never be sure if the compiler detects possible vectorizations. So in this class we will try to always vectorize our code manually. 
 <strong>As a general rule, you should avoid tight loops at all cost.</strong>



<h4> How to program in Julia / Matlab / Numpy </h4>

<p>Although there is an execution overhead per line in Julia, matrix operations are extremely optimized and very fast. In order to become a successful Julia programmer, you need to free yourself from the "for-loop" thinking and start thinking in terms of matrix operations. Julia can be very fast, if almost all the time is spent in a few heavy duty matrix operations. In this assignment you will do this, and transform the function above into a few matrix operations <em>without any loops at all.</em> </p> 

<p>The key to efficient programming in Julia and machine learning in general is to think about it in terms of mathematics, and not in terms of Loops. </p>

<p>	(a) Show that the Gram matrix (aka inner-product matrix)
$$	G_{ij}=\vec x_i^\top\vec z_j $$
can be expressed in terms of a pure matrix multiplication. Once you are done with the derivation, implement the function <strong><code>innerproduct</code></strong>.</p>


In [3]:
#<GRADED>
function innerproduct(X,Z=[])
    # function innerproduct(X,Z)
    #
    # Computes the inner-product matrix.
    # Syntax:
    # D=innerproduct(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix G of size nxm
    # G(i,j) is the inner-product between vectors X(:,i) and Z(:,j)
    #
    # call with only one input:
    # innerproduct(X)=innerproduct(X,X)
    #

    if (isempty(Z)) # case when there is only one input (X)
    ## fill in code here

    else  # case when there are two inputs (X,Z)
    ## fill in code here 

    end;
    return(G)
end
#</GRADED>

If your code is correct you should pass the following two tests below. 

In [4]:
# a simple test for the innerproduct function
function innertest()
    M=reshape(1:9,3,3)';
    Q=reshape(11:16,3,2)';
    println(norm(innerproduct(M,M)-innerproduct(M))<1e-16) # test1: Inner product with itself
    println(innerproduct(M,Q)==[[74;182;290] [92;227;362]])
end
innertest() # function call

LoadError: LoadError: UndefVarError: innerproduct not defined
while loading In[4], in expression starting on line 8

(b) Let us define two new matrices $S,R\in{\cal R}^{n\times m}$ 
		$$S_{ij}=\vec x_i^\top\vec x_i, \ \ R_{ij}=\vec z_j^\top\vec z_j.$$
 	Show that the <em>squared</em>-euclidean matrix $D^2\in{\cal R}^{n\times m}$, defined as
		$$D^2_{ij}=(\vec x_i-\vec z_j)^2,$$
	can be expressed as a linear combination of the matrix $S, G, R$. (Hint: It might help to first express $D^2_{ij}$ in terms of inner-products.) What do you need to do to obtain the true Euclidean distance matrix $D$?</p></td>
	
<p>

<p>	(c) Implement the function <strong><code>l2distance</code></strong>, which computes the Euclidean distance matrix $D$ without a single loop:

In [5]:
#<GRADED>
function l2distance(X,Z=[])
    # function D=l2distance(X,Z)
    #
    # Computes the Euclidean distance matrix.
    # Syntax:
    # D=l2distance(X,Z)
    # Input:
    # X: nxd data matrix with n vectors (rows) of dimensionality d
    # Z: mxd data matrix with m vectors (rows) of dimensionality d
    #
    # Output:
    # Matrix D of size nxm
    # D(i,j) is the Euclidean distance of X(:,i) and Z(:,j)
    #
    # call with only one input:
    # l2distance(X)=l2distance(X,X)
    #

    if(isempty(Z));
        ## fill in code here

    else  # case when there are two inputs (X,Z)
            ## fill in code here

    end
    return(D);
end
#</GRADED>

Test your l2-distance function again:

In [1]:
function timecomparison()
    X=rand(700,100);
    Z=rand(300,100);
    println("Running the naive version ...")
    tic();D=l2distanceSlow(X,Z);tslow=toc()
    print D
    println("Running the vectorized version ...")
    tic();D=l2distance(X,Z);tfast=toc()
    speedup=tslow/tfast;
    @printf "The vectorized method is %ix faster!" speedup
end
timecomparison() # function call

LoadError: LoadError: syntax: extra token "D" after end of expression
while loading In[1], in expression starting on line 6

How much faster is your code now? With this implementation you should easily be able to compute the distances between <strong>many more</strong> vectors. 
Call the following code to see how many distance computations you can perform within one second. With the loopy implementations, the same computation might have taken you several days. </p>